In [1]:
# Imports
load('prismatic_envelope.sage')
load('precision.py')

### User-defined input

In [2]:
# The prime p
p=2

# The motivic weight i in F_p(i)^red
i=2

# The power of the uniformizer n
n=2

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [3]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

### From renormalizing the Eisenstein polynomial
precision_loss_from_Eisenstein=1

### Probably this precision can be taken to be lower since we will only need the
### Fp-coefficient calculation
total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives
                 +precision_loss_from_Eisenstein)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 10
Fprec is 4


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [4]:
# The Eisenstein polynomial E
E=A(z+p)

### The main calculation

In [5]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

if i-p+1>0:
    a_syn0,a_syn1,a_nablaN,a_nablaP,b_syn0,b_syn1,b_nablaN,b_nablaP,v1N0,v1P0,v1N1,v1P1=syntomic_matrices_v1(p,i,n,E,total_precision,Fprec,debug=False)

[10]
[100   0   0]
[100  10   0]
[  1   1  10]
[0]
[0]
[1]


<ipython-input-5-3a62c072849d>:9: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See http://trac.sagemath.org/14825 for details.
  a_syn0,a_syn1,a_nablaN,a_nablaP,b_syn0,b_syn1,b_nablaN,b_nablaP,v1N0,v1P0,v1N1,v1P1=syntomic_matrices_v1(p,i,n,E,total_precision,Fprec,debug=False)


### Assembling the syntomic complex and computing its cohomology

In [14]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(a_syn0,a_syn1,a_nablaN,a_nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*(i-p+1)-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*(i-p+1)-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_0(R;Z_p) are []
Elementary divisors of K_1(R;Z_p) are [10]
Target precision is 3 and final precision is 9


In [15]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(b_syn0,b_syn1,b_nablaN,b_nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_2(R;Z_p) are [10]
Elementary divisors of K_3(R;Z_p) are [1000]
Target precision is 3 and final precision is 4


In [100]:
a_d0=block_matrix([[a_syn0],[a_nablaN]])
a_d1=block_matrix([[a_nablaP,-a_syn1]])
a_d0_Fp=Matrix(GF(p),a_d0)
a_d1_Fp=Matrix(GF(p),a_d1)
a_C=ChainComplex({0:a_d0_Fp,1:a_d1_Fp})
a_C.homology()

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 1 over Finite Field of size 2,
 2: Vector space of dimension 0 over Finite Field of size 2}

In [101]:
b_d0=block_matrix([[b_syn0],[b_nablaN]])
b_d1=block_matrix([[b_nablaP,-b_syn1]])
b_d0_Fp=Matrix(GF(p),b_d0)
b_d1_Fp=Matrix(GF(p),b_d1)
b_C=ChainComplex({0:b_d0_Fp,1:b_d1_Fp})
b_C.homology()

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 2 over Finite Field of size 2,
 2: Vector space of dimension 1 over Finite Field of size 2}

In [106]:
b_d0

[     100        0        0]
[      11       10        0]
[       1        1       10]
[--------------------------]
[      10        0        0]
[10101011  1010110        0]
[10001100  1110001    11110]

In [44]:
M.kernel?

In [49]:
M.left_kernel(basis='pivot')

Vector space of degree 6 and dimension 4 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0]
[0 1 0 0 1 0]
[0 0 1 0 1 1]
[0 0 0 1 0 0]

In [48]:
M.right_kernel(basis='pivot')

Vector space of degree 3 and dimension 1 over Finite Field of size 2
Basis matrix:
[0 0 1]

In [47]:
M.kernel?

In [51]:
M.smith_form()

(
[1 0 0]  [0 1 0 0 0 0]         
[0 1 0]  [0 1 1 0 0 0]         
[0 0 0]  [1 0 0 0 0 0]         
[0 0 0]  [0 0 0 1 0 0]  [1 0 0]
[0 0 0]  [0 1 0 0 1 0]  [0 1 0]
[0 0 0], [0 1 1 0 0 1], [0 0 1]
)

In [54]:
M.smith_form?

In [55]:
a_C

Chain complex with at most 3 nonzero terms over Finite Field of size 2

In [56]:
b_C

Chain complex with at most 3 nonzero terms over Finite Field of size 2

In [57]:
ascii_art(a_C)

                        [0]      
            [1 1]       [0]      
 0 <-- C_2 <------ C_1 <---- C_0 <-- 0 

In [58]:
ascii_art(b_C)

                                [0 0 0]      
                                [1 0 0]      
                                [1 1 0]      
            [1 0 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [1 0 0]      
            [0 0 1 0 1 1]       [0 1 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [97]:
v1_0=Matrix(GF(p),v1N0)
v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
v1_2=Matrix(GF(p),v1P1)

In [107]:
ascii_art(block_matrix([[v1P0,0],[0,v1N1]]))

[         0|         0]
[         0|         0]
[1110001111|         0]
[----------+----------]
[         0|         0]
[         0|         0]
[         0|         1]

In [96]:
ascii_art(v1_1)

[0 0]
[0 0]
[1 0]
[0 0]
[0 0]
[0 1]

In [98]:
B=a_C.differential(0).right_kernel().basis_matrix().transpose()
A=b_C.differential(0).right_kernel().basis_matrix().transpose()
S,U,V=A.smith_form()
r=S.rank()
print(V.dimensions())
print(standard_projection_matrix(r,i*n-1).dimensions())
print(U.dimensions())
print(v1_0.dimensions())
print(B.dimensions())
v1_H0=V*Matrix(GF(p),standard_projection_matrix(r,i*n-1))*U*v1_0*B

(1, 1)
(1, 3)
(3, 3)
(3, 1)
(1, 1)


In [138]:
def kernel_image_factorization(M):
    # Returns A,B where A is the projection onto the image and B is the inclusion of the image.
    S,U,V=M.smith_form()
    print(S.dimensions())
    print(U.dimensions())
    print(V.dimensions())
    r=S.rank()
    P=standard_projection_matrix(r,M.dimensions()[1])
    Q=standard_projection_matrix(r,M.dimensions()[0]).transpose()
    print(P.dimensions())
    print(Q.dimensions())
    return P*V^(-1),U^(-1)*Q

In [139]:
v1_1

[0 0]
[0 0]
[1 0]
[0 0]
[0 0]
[0 1]

In [141]:
A,B=kernel_image_factorization(v1_1)

(6, 2)
(6, 6)
(2, 2)
(2, 2)
(6, 2)


In [142]:
B*A

[0 0]
[0 0]
[1 0]
[0 0]
[0 0]
[0 1]

In [117]:
A=a_C.differential(1)
B=b_C.differential(1)
S,U,V=A.smith_form()
r=S.rank()
s=A.dimensions()[0]
X=block_matrix([[Matrix(GF(p),r,s-r)],[identity_matrix(s-r)]])
Y=U^(-1)*X
S,U,V=B.smith_form()
r=S.rank()
s=B.dimensions()[0]
Z=block_matrix([[Matrix(GF(p),r,s-r)],[identity_matrix(s-r)]]).transpose()
print(Z.dimensions())
print(v1_2.dimensions())
print(Y.dimensions())
v1_H2=Z*v1_2*Y

(1, 3)
(3, 1)
(1, 0)


In [119]:
v1_H2.dimensions()

(1, 0)

In [77]:
standard_projection_matrix(1,3)

[1 0 0]

In [85]:
M.dimensions?

In [86]:
A.smith_form()

(
              [0 0 1]
              [0 1 0]
[1 0 0], [1], [1 0 0]
)

In [88]:
A=b_C.differential(0).right_kernel().basis_matrix();A

[0 0 1]

In [89]:
A,smith_form?

Object `smith_form` not found.


In [90]:
A.smith_form?

In [91]:
A.smith_form()

(
              [0 0 1]
              [0 1 0]
[1 0 0], [1], [1 0 0]
)

In [109]:
Y=Matrix(ZZ,5,5)

In [110]:
Y

[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In [122]:
A=standard_projection_matrix(2,3)
B=A.transpose()

In [123]:
B

[1 0]
[0 1]
[0 0]

In [124]:
A*B

[1 0]
[0 1]

In [125]:
B*A

[1 0 0]
[0 1 0]
[0 0 0]

In [143]:
a_C

Chain complex with at most 3 nonzero terms over Finite Field of size 2

In [145]:
ascii_art(b_C)

                                [0 0 0]      
                                [1 0 0]      
                                [1 1 0]      
            [1 0 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [1 0 0]      
            [0 0 1 0 1 1]       [0 1 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [377]:
def complex_smith_form(C):
    # Returns a triple (D,f,g) where each differential is in ``Smith form''
    # and f is a map of chain complexes C -> D.
    # Assumes that the degree of the differential is +1.
    nz=C.nonzero_degrees()
    deg_min=min(nz)
    deg_max=max(nz)
    # The number of differentials
    lt=deg_max-deg_min
    f_dict={}
    g_dict={}
    for i in range(deg_min,deg_max+1):
        f_dict[i]=identity_matrix(C.free_module_rank(i))
        g_dict[i]=identity_matrix(C.free_module_rank(i))
    D_dict={}
    j=0
    S,U,V=C.differential()[deg_max-(j+1)].smith_form()
    D_dict[deg_max-(j+1)]=S
    f_dict[deg_max-j]=U*f_dict[deg_max-j]
    f_dict[deg_max-(j+1)]=V^(-1)*f_dict[deg_max-(j+1)]
    g_dict[deg_max-j]=g_dict[deg_max-j]*U^(-1)
    g_dict[deg_max-(j+1)]=g_dict[deg_max-(j+1)]*V    
    for j in range(1,lt):
        new_d=f_dict[deg_max-j]*C.differential()[deg_max-(j+1)]
        col_offset=0
        row_offset=D_dict[deg_max-j].transpose().rank()
        col_num=C.differential()[deg_max-(j+1)].dimensions()[1]
        row_num=D_dict[deg_max-j].transpose().nullity()
        new_d_sub=new_d.submatrix(row_offset,col_offset,row_num,col_num)
        S_sub,U_sub,V=new_d_sub.smith_form()
        S=block_matrix([[Matrix(row_offset,col_num)],[S_sub]])
        U=block_matrix([[identity_matrix(row_offset),0],[0,U_sub]])
        f_dict[deg_max-j]=U*f_dict[deg_max-j]
        f_dict[deg_max-(j+1)]=V^(-1)*f_dict[deg_max-(j+1)]
        g_dict[deg_max-j]=g_dict[deg_max-j]*U^(-1)
        g_dict[deg_max-(j+1)]=g_dict[deg_max-(j+1)]*V
        D_dict[deg_max-(j+1)]=S
    return ChainComplex(D_dict),f_dict,g_dict

def homology_smith_form(D,i):
    nz=D.nonzero_degrees()
    deg_min=min(nz)
    deg_max=max(nz)
    if i<deg_min or i>deg_max:
        return []
    elif i==deg_max:
        num_gens=D.differential()[i-1].nrows()
        offset=0
        l=[]
        for j in range(num_gens):
            if j<D.differential()[i-1].ncols() and j+offset<D.differential()[i-1].nrows():
                div=D.differential()[i-1][j+offset,j]
                if not div.is_unit():
                    l.append(div)
            else:
                l.append(0)
        return l
    elif i>deg_min:
        num_gens=D.differential()[i].ncols()-D.differential()[i].rank()
        l=[]
        offset=D.differential()[i].rank()
        for j in range(num_gens):
            if j<D.differential()[i-1].ncols() and j+offset<D.differential()[i-1].nrows():
                div=D.differential()[i-1][j+offset,j]
                if not div.is_unit():
                    l.append(div)
            else:
                l.append(0)
        return l
    elif i==deg_min:
        num_gens=D.differential()[i].ncols()-D.differential()[i].rank()
        l=[0]*num_gens
        return l
    
def morphism_homology(A,A_smith,fA,gA,B,B_smith,fB,gB,F,i):
    new_F=fB[i]*F[i]*gA[i]
    x=len(homology_smith_form(A_smith,i))
    y=len(homology_smith_form(B_smith,i))
    return new_F.submatrix(new_F.nrows()-y,new_F.ncols()-x)

In [387]:
v1={0:v1_0,1:v1_1,2:v1_2}
a_C_smith,f_a,g_a=complex_smith_form(a_C)
b_C_smith,f_b,g_b=complex_smith_form(b_C)
morphism_homology(a_C,a_C_smith,f_a,g_a,b_C,b_C_smith,f_b,g_b,v1,1)

[0]
[0]

In [388]:
ascii_art(a_C_smith)

                        [0]      
                        [-]      
            [1 0]       [0]      
 0 <-- C_2 <------ C_1 <---- C_0 <-- 0 

In [389]:
ascii_art(b_C_smith)

                                [0 0 0]      
                                [0 0 0]      
                                [-----]      
                                [1 0 0]      
            [1 0 0 0 0 0]       [0 1 0]      
            [0 1 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [0 0 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [390]:
ascii_art(a_C)

                        [0]      
            [1 1]       [0]      
 0 <-- C_2 <------ C_1 <---- C_0 <-- 0 

In [391]:
ascii_art(b_C)

                                [0 0 0]      
                                [1 0 0]      
                                [1 1 0]      
            [1 0 0 0 0 0]       [0 0 0]      
            [0 0 0 0 0 0]       [1 0 0]      
            [0 0 1 0 1 1]       [0 1 0]      
 0 <-- C_2 <-------------- C_1 <-------- C_0 <-- 0 

In [392]:
v1[0]

[0]
[0]
[1]

In [393]:
v1[1]

[0 0]
[0 0]
[1 0]
[0 0]
[0 0]
[0 1]

In [394]:
v1[2]

[0]
[0]
[1]

In [396]:
v1[1]*a_C.differential()[0]==b_C.differential()[0]*v1[0]

True

In [397]:
v1[2]*a_C.differential()[1]==b_C.differential()[1]*v1[1]

True

In [ ]:
f